In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
# Python < 3.8?
import sys
#! pip install pickle5

if sys.version_info.minor < 8:
    import pickle5 as pickle

In [4]:
import sys
sys.path.append("../")
from registration_tree import Rect, QuadTree

In [25]:
qtrees = {
    0: {},
    1: {},
    2: {},
    3: {},
}
quality = []

for path in Path("../").glob("*/*/*/*-To-*.pickle"):
    
    source, target = path.stem.split("-To-")
    
    qtree = pickle.load(open(str(path), "rb" ))
    
    if source not in qtrees[qtree.target_depth]:
        qtrees[qtree.target_depth][source] = {}
    
    patient_id = source.split("_")[-2]
    image_type = "CCMCT" if "CCMCT" in str(path) else "Cyto"
    quality.append([patient_id, source, target, int(qtree.q), int(qtree.sigma2), 
                    image_type, qtree.target_depth, qtree.b])
    
    qtrees[qtree.target_depth][source][target] = qtree   
    
quality = pd.DataFrame(quality, columns=["patient_id", "source", "target","q", "sigma2", "image_type", "target_depth", "b"])
quality.sort_values(["patient_id", "target_depth"])

,patient_id,source,target,q,sigma2,image_type,target_depth,b
0,183715A,A_CCMCT_183715A_1,N1_CCMCT_183715A_1,576,1229,CCMCT,0,"[[1.147262496221766, -0.001589742235138383], [..."
1,183715A,A_CCMCT_183715A_1,N2_CCMCT_183715A_1,360,578,CCMCT,0,"[[1.117189506727029, 0.0030364051921110587], [..."
2,183715A,A_CCMCT_183715A_1,Z_CCMCT_183715A_1,479,1087,CCMCT,0,"[[1.1488010533086592, 0.00609033201719216], [-..."
15,183715A,A_CCMCT_183715A_1,N1_CCMCT_183715A_1,576,1229,CCMCT,1,"[[1.147262496221766, -0.001589742235138383], [..."
16,183715A,A_CCMCT_183715A_1,N2_CCMCT_183715A_1,360,578,CCMCT,1,"[[1.117189506727029, 0.0030364051921110587], [..."
...,...,...,...,...,...,...,...,...
64,574162,A_BB_574162_1,N1_BB_574162_1,522,171,Cyto,1,"[[1.1474815918883285, -0.0010241465659416424],..."
65,574162,A_BB_574162_1,N2_BB_574162_1,362,128,Cyto,1,"[[1.1181893148835758, 0.0035647499956651873], ..."
66,574162,A_BB_574162_1,Z_BB_574162_1,438,163,Cyto,1,"[[1.1489408206538754, 0.011306338118543953], [..."
76,574162,A_BB_574162_1,N2_BB_574162_1,362,128,Cyto,2,"[[1.1181893148835758, 0.0035647499956651873], ..."


In [12]:
annotations = pd.read_csv("GT.csv")

annotations["image_name_stem"] = [Path(image_name).stem for image_name in annotations["image_name"]]

annotations["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in annotations["vector"]]
annotations["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in annotations["vector"]]

annotations["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in annotations["vector"]]
annotations["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["center"] = [np.array((center_x, center_y)) for center_x, center_y in zip(annotations["center_x"], annotations["center_y"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,image_name_stem,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,...,N2_CCMCT_380609B_1,20354,16752,20408,16806,20381.0,16779.0,"[20381.0, 16779.0]",54,54
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,...,N2_CCMCT_380609B_1,20256,35448,20310,35502,20283.0,35475.0,"[20283.0, 35475.0]",54,54
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,...,N2_CCMCT_380609B_1,20445,54139,20499,54193,20472.0,54166.0,"[20472.0, 54166.0]",54,54
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,...,N2_CCMCT_380609B_1,26495,71854,26549,71908,26522.0,71881.0,"[26522.0, 71881.0]",54,54
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,...,N2_CCMCT_380609B_1,35443,90091,35497,90145,35470.0,90118.0,"[35470.0, 90118.0]",54,54


In [18]:
results, quality = [], []

for path in Path("../").glob("*/*/*/*-To-*.pickle"):
    
    source_name, target_name = path.stem.split("-To-")
    
    qtree = pickle.load(open(str(path), "rb" ))
    
    patient_id = source_name.split("_")[-2]
    image_type = "CCMCT" if "CCMCT" in str(path) else "Cyto"
    
    quality.append([patient_id, source_name, target_name, int(qtree.mean_reg_error), qtree.thumbnail_size[0], 
                    image_type, qtree.target_depth, qtree.run_time])
    
    
    source_annos = annotations[(annotations["image_name_stem"] == source_name) & 
                                   (annotations["image_type"] == image_type)]
    
    target_anno = annotations[(annotations["image_name_stem"] == target_name) & 
                                   (annotations["image_type"] == image_type)].iloc[0]
    
    target_scanner = target_anno.scanner
    image_id = target_anno.image_id
    image_name = target_anno.image_name
    image_width, image_height = target_anno.image_width, target_anno.image_height
    
    for id, source_anno in source_annos.iterrows():

        box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]

        trans_box = qtree.transform_boxes(np.array([box]))[0]

        new_x1 = int(trans_box[0] - trans_box[2] // 2)
        new_y1 = int(trans_box[1] - trans_box[3] // 2)
        new_x2 = int(trans_box[0] + trans_box[2] // 2)
        new_y2 = int(trans_box[1] + trans_box[3] // 2)

        vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

        row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                       vector, source_anno.unique_identifier, source_anno.annotation_type, 
                       source_anno.type_name, f"QTree_L{qtree.target_depth}", qtree.run_time, qtree.thumbnail_size[0]]

        results.append(row)
        
quality = pd.DataFrame(quality, columns=["patient_id", "source", "target","mean_reg_error", "thumbnail_size", "image_type", "target_depth", "run_time"])
        
results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", 
                                         "type_name", "method", "runtime", "thumbnail_size"])
results.head()

,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method,runtime,thumbnail_size
0,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,"{'x1': 15437, 'x2': 15485, 'y1': 9464, 'y2': 9...",45e2d88d-8211-419b-a754-6d71dd317ec5,384,L0,QTree_L0,0.795544,1024
1,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,"{'x1': 10295, 'x2': 10343, 'y1': 20158, 'y2': ...",ff307a78-11ec-4a05-b8c8-ad90a2cb18ed,385,L1,QTree_L0,0.795544,1024
2,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,"{'x1': 9130, 'x2': 9178, 'y1': 32877, 'y2': 32...",ee578e27-99c4-4ab9-a10a-0a19fd58f4fa,386,L2,QTree_L0,0.795544,1024
3,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,"{'x1': 9261, 'x2': 9309, 'y1': 44434, 'y2': 44...",27f6e3b9-8627-414a-b6d8-b8e111031d54,387,L3,QTree_L0,0.795544,1024
4,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,"{'x1': 24119, 'x2': 24167, 'y1': 8310, 'y2': 8...",212779a8-3d64-4ea3-a604-a7f775bb4d0f,389,L5,QTree_L0,0.795544,1024


In [29]:
for thumbnail_size in results["thumbnail_size"].unique():
    
    data = results[results["thumbnail_size"] == thumbnail_size]
    results.to_csv(f"QuadTree_{thumbnail_size}.csv", index=False)

In [20]:
quality.head()

,patient_id,source,target,mean_reg_error,thumbnail_size,image_type,target_depth,run_time
0,183715A,A_CCMCT_183715A_1,A_CCMCT_183715A_1,0,1024,CCMCT,0,0.795544
1,183715A,A_CCMCT_183715A_1,N1_CCMCT_183715A_1,17,1024,CCMCT,0,0.686660
2,183715A,A_CCMCT_183715A_1,N2_CCMCT_183715A_1,36,1024,CCMCT,0,0.558358
3,183715A,A_CCMCT_183715A_1,Z_CCMCT_183715A_1,29,1024,CCMCT,0,0.761948
4,22108,A_CCMCT_22108_1,A_CCMCT_22108_1,0,1024,CCMCT,0,1.284026


In [28]:
"""
for patient_id in quality["patient_id"]:
    
    patient_data = quality[(quality["patient_id"] == patient_id) & ~(quality["source"] == quality["target"])]
    
    fig, ax = plt.subplots(figsize=(24, 8))

    ax.set_title(f"{patient_id}")
    
    for thumbnail_size in patient_data["thumbnail_size"]:
        for target_depth in patient_data["target_depth"]:
            
            line_data = patient_data[(patient_data["thumbnail_size"] == thumbnail_size) & 
                                     (patient_data["target_depth"] == target_depth)]

            sns.lineplot(x="mean_reg_error", y="run_time",
                         hue="thumbnail_size",  #scanner
                         #style="target_depth",
                         data=line_data, ax=ax)

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    
    break
"""

'\nfor patient_id in quality["patient_id"]:\n    \n    patient_data = quality[(quality["patient_id"] == patient_id) & ~(quality["source"] == quality["target"])]\n    \n    fig, ax = plt.subplots(figsize=(24, 8))\n\n    ax.set_title(f"{patient_id}")\n    \n    for thumbnail_size in patient_data["thumbnail_size"]:\n        for target_depth in patient_data["target_depth"]:\n            \n            line_data = patient_data[(patient_data["thumbnail_size"] == thumbnail_size) & \n                                     (patient_data["target_depth"] == target_depth)]\n\n            sns.lineplot(x="mean_reg_error", y="run_time",\n                         hue="thumbnail_size",  #scanner\n                         #style="target_depth",\n                         data=line_data, ax=ax)\n\n    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)\n    plt.show()\n    \n    break\n'